In [15]:
import shapely
import geopandas
import pathlib
import OSMPythonTools
import OSMPythonTools.api
import OSMPythonTools.overpass

In [11]:
basepath = pathlib.Path(r"C:\Users\pearsonra\Documents\data\drain_detection\catchments")
crs=2193

# Functions

In [22]:
def create_catchment(x0: float, y0: float, x1: float, y1: float, name:str) -> geopandas.GeoDataFrame:
    catchment = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0, y0], [x1, y0], [x1, y1], [x0, y1]])], crs=crs)
    catchment_path = (basepath / name)
    catchment_path.mkdir(parents=True, exist_ok=True)
    catchment.to_file(catchment_path / "catchment.geojson")
    return catchment

# Kiaipoi
Create and save a catchment file and download and save drains in the catchment

In [18]:
name = "kiaipoi"
x0 = 1556700
y0 = 5204450
x1 = 1571700
y1 = 5193450

Create and save catchment

In [24]:
catchment = create_catchment(x0=x0, y0=y0, x1=x1, y1=y1, name=name)

Query OSM

In [ ]:
def download_drains(catchment: geopandas.GeoDataFrame) -> geopandas.GeoDataFrame:
    bbox_lat_long = catchment.to_crs('EPSG:4326').bounds
    
    query = overpassQueryBuilder(bbox=[bbox_lat_long.miny[0],
                                   bbox_lat_long.minx[0],
                                   bbox_lat_long.maxy[0],
                                   bbox_lat_long.maxx[0]], 
                             elementType='way', selector='waterway',
                             out='body', includeGeometry=True)

In [ ]:
bbox_lat_long = bbox_nztm2000.to_crs('EPSG:4326')
query = overpassQueryBuilder(bbox=[bbox_lat_long.bounds.miny[0],
                                   bbox_lat_long.bounds.minx[0],
                                   bbox_lat_long.bounds.maxy[0],
                                   bbox_lat_long.bounds.maxx[0]], 
                             elementType='way', selector='waterway',
                             out='body', includeGeometry=True)
overpass = OSMPythonTools.overpass.Overpass()
rivers = overpass.query(query)
element_dict = {
    'geometry': [],
    'OSM_id': [],
    'waterway': [],
    'tunnel': [],
}

for element in rivers.elements():
    element_dict['geometry'].append(element.geometry())
    element_dict['OSM_id'].append(element.id())
    element_dict['waterway'].append(element.tags()['waterway'])
    element_dict['tunnel'].append('tunnel' in element.tags().keys())
drains_df = geopandas.GeoDataFrame(element_dict, crs='EPSG:4326').to_crs('EPSG:2193')

drain_width = 5
drain_polygon = drains_df[drains_df.geometry.type == "LineString"].buffer(drain_width)
drain_polygon = geopandas.GeoDataFrame(
    geometry=[shapely.ops.unary_union(drain_polygon.geometry.array)],
    crs=drain_polygon.crs,
)
drain_polygon = drain_polygon.clip(bbox_nztm2000)
drain_polygon.plot()

drains = drains[(drains['waterway'] != 'river') ]

open_drains = drains[(drains['tunnel'] == False)]
closed_drains = drains[drains['tunnel']]